In [1]:
import numpy as np, matplotlib.pyplot as plt, random, json, pickle, datetime, copy, socket, os
from scipy.stats import sem
import matplotlib.colors as colors
from scipy.ndimage import gaussian_filter as gauss # for smoothing ratemaps
import sys
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as GS
from matplotlib.backends.backend_pdf import PdfPages
from importlib import reload

if socket.gethostname() == 'Tolman':
    codeDirBase = 'C:\\Users\\whockei1\\Google Drive'
elif socket.gethostname() == 'DESKTOP-BECTOJ9':
    codeDirBase = 'C:\\Users\\whock\\Google Drive'
    
sys.path.insert(0, codeDirBase + '\\KnierimLab\\Ratterdam\\Code')
sys.path.insert(0, codeDirBase + '\\Python_Code\\KLab\\mts_analysis')
import utility_fx as util
import ratterdam_ParseBehavior as pBehav
import ratterdam_CoreDataStructures as core

In [2]:
from ratterdam_Defaults import *

In [3]:
datafile = "E:\\Ratterdam\\R765\\R765DFD4\\"
clust = "TT14\\cl-maze1.5"

In [2]:
%qtconsole --style native

In [899]:
pos = util.read_pos(datafile)
clust = util.read_clust(datafile+clust)
clust = np.asarray(clust)
ts = np.asarray(sorted(list(pos.keys())))
posx = [640 - pos[i][0] for i in ts]
posy = [pos[i][1] for i in ts]
for t in ts:
    pos[t][0] = 640 - pos[t][0]
position = np.column_stack((ts,posx,posy))
spikexy = util.getPosFromTs(clust,position)
spikes = np.column_stack((clust,spikexy))
spikes = spikes[np.where(spikes[:,2] > 50)]
position = position[np.where(position[:,2] > 50)]
alleyTracking, alleyVisits,  txtVisits = pBehav.getDaysBehavioralData(datafile)

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [11]:
def alleyData(alley,array):
    '''get all spikes/pos in an alley'''
    alley = alley-1
    z = array[
        (array[:,1] > alleyBounds[alley][0][0]) & \
        (array[:,1] < alleyBounds[alley][0][1]) & \
        (array[:,2] > alleyBounds[alley][1][0]) & \
        (array[:,2] < alleyBounds[alley][1][1])  
                       ]
    return z

def findTxt(ts,alley):
    '''what txt was present at a given '''
    return stimData['stimuli'][alley-1][bisect(alleySwapTS[alley-1],ts/1e6)-1] # bisect finds where to put this ts in list of txt swaps
                                                                       # then look up the previous entry in txt list, i.e. one present during that ts

In [12]:
def alleyRatemapData(alley,txt, allyVisits):
    '''basic unit of analysis. get all visits by a txt
    Return array of spikes and pos for that txt/alley combo '''
    alley = alley-1
    alleyPos = np.empty((0,3))
    alleySpikes = np.empty((0,3))
    visitidx = np.empty((0,2))
    for i in range(len(alleyVisits[alley])):
        if txtVisits[alley][i] == txt:
            visitsOcc = util.getVisitPos(alley, i, ts, alleyVisits, position)
            visitsSpk = util.getVisitPos(alley, i, spikes[:,0],alleyVisits, position)
            alleyPos = np.vstack((alleyPos, visitsOcc))
            alleySpikes = np.vstack((alleySpikes, visitsSpk))
            visitidx = np.vstack((visitidx, (visitsSpk.shape[0],visitsOcc.shape[0])))
    spkI = np.cumsum(visitidx[:,0]).astype(int)
    posI = np.cumsum(visitidx[:,1]).astype(int)
    return alleySpikes, alleyPos, np.column_stack((spkI,posI))

In [13]:
allTrackbyTxt = {'A':{'spk':np.empty((0,3)),'pos':np.empty((0,3))},
                 'B':{'spk':np.empty((0,3)),'pos':np.empty((0,3))},
                 'C':{'spk':np.empty((0,3)),'pos':np.empty((0,3))}}

allTrack = {'spk':np.empty((0,3)),'pos':np.empty((0,3))}
alleyByTxt = {i:{'A':{'spk':None,'pos':None,'idx':None},
                 'B':{'spk':None,'pos':None,'idx':None},
                 'C':{'spk':None,'pos':None,'idx':None}}
                for i in range(1,18)}

for i in range(1,18):
    for txt in ['A','B','C']:
        s, p, x = alleyRatemapData(i,txt, alleyVisits)
        
        for entry,data in zip(['spk','pos','idx'],[s,p,x]):
            alleyByTxt[i][txt][entry] = data
    
        for entry, data in zip(['spk','pos'],[s,p]):
            allTrackbyTxt[txt][entry] = np.vstack((allTrackbyTxt[txt][entry],data))
            allTrack[entry] = np.vstack((allTrack[entry], data))

In [568]:
def weird_smooth(U,sigma):
    V=U.copy()
    V[U!=U]=0
    VV=sp.ndimage.gaussian_filter(V,sigma=2.0)

    W=0*U.copy()+1
    W[U!=U]=0
    WW=sp.ndimage.gaussian_filter(W,sigma=2.0)

    Z=VV/WW
    return Z

def get_long_alley_dim():
    '''all alleys should have same dim, as theyre same size. so use alley  1 by convention'''
    r,c = alleyBins[0]['rows'].shape[0], alleyBins[0]['cols'].shape[0]
    return max(r,c)

def getAxType(array):
    '''should "axis" = 1 or 2? Which dim is longer'''
    if array.shape[0] > array.shape[1]:
        return 1
    elif array.shape[0] < array.shape[1]:
        return 0
    
def linHist(alley,spikes,position):
    '''1-indexed alley. Using [arr, arr] for bins from alleyBins'''
    rbins,cbins = alleyBins[alley-1]['rows'], alleyBins[alley-1]['cols']
    if rbins.shape[0] < cbins.shape[0]:
        hs = np.histogram2d(spikes[:,1],spikes[:,2],bins=[cbins,rbins])[0]
        ho = np.histogram2d(position[:,1],position[:,2],bins=[cbins,rbins])[0]
    else:
        hs = np.histogram2d(spikes[:,2],spikes[:,1],bins=[rbins,cbins])[0]
        ho = np.histogram2d(position[:,2],position[:,1],bins=[rbins,cbins])[0]
    ls,lo  = np.sum(hs,axis=1), np.sum(ho,axis=1)
    return ls,lo

def linearizeVisits(alley, txt):
    sv, pv = getVisits(alley, txt)
    ls_trials, lo_trials = np.empty((0,get_long_alley_dim()-1)), np.empty((0,get_long_alley_dim()-1)) 
    for _s, _p in zip(sv,pv):
        ls,lo = linHist(alley,_s,_p)
        ls_trials, lo_trials = np.vstack((ls_trials, ls)), np.vstack((lo_trials, lo))
    return ls_trials, lo_trials

def linearizeAlley(alley,txt,**kwargs):
    '''Given spike/pos arrays corresponding to all data in alley
    and array of indices separating different visits, linearize occ norm
    visit and return all visits (opts='both') or just visits/mean (opt='visits'/'mean')
    1st col of idx is spk visit indx, 2nd is pos visit idx
    '''
    nanpol = kwargs['nanpol']
    ls_trials, lo_trials = linearizeVisits(alley, txt)
    n = (np.sum(ls_trials,axis=0) * np.reciprocal(np.sum(lo_trials,axis=0)) ) * 33  
    n = weird_smooth(n,2)
    nonOccupancyPos = np.where(~lo_trials.any(axis=0))[0]
    if nanpol == 'nan':
        n[nonOccupancyPos] = np.nan
    elif nanpol == 'zero':
        n[nonOccupancyPos] = 0
    return n

def getVisits(alley,txt):
    spikes, pos, idx = alleyByTxt[alley][txt]['spk'],alleyByTxt[alley][txt]['pos'],alleyByTxt[alley][txt]['idx']
    sv, pv = np.split(spikes,idx[:,0]), np.split(pos,idx[:,1]) # list of subarrays
    return sv[:-1], pv[:-1]

def viewVisits(alley,txt, viewType):
    '''options are "imshow" or "linear"'''
    if viewType == 'imshow':
        rms = linearizeAlley(alley,txt,**{'nanpol':'nan'})
        fig = plt.figure()
        ax = fig.add_subplot(111)
        im = ax.imshow(rms, cmap=mycm)
        ax.set_title(f"Alley {alley}, Texture {txt}")
        cbar_ax = fig.add_axes([0.95, 0.15, 0.05, 0.7])
        fig.colorbar(im, cax=cbar_ax,extend='max')
    elif viewType == 'linear':
        rms = linearizeAlley(alley, txt, **{'nanpol':'zero'})
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_title(f"Alley {alley}, Texture {txt}")
        for i in rms:
            plt.plot(i)

In [85]:
def list_to_arrays(l):
    arr = np.empty((0,l[0].shape[1]))
    for subarray in l:
        arr = np.vstack((arr,subarray))
    return arr

In [418]:
def computeSingleRM(spikes,position,alley):
    '''Compute a single rate map from an array of spikes
    and pos of array form (ts,x,y). Not using track coords, just 
    0-max bin size in either array. Chooses which dim is long
    depending on which alley (lookup in fx)'''
    #So np.hist2d takes bins in [x,y] which is [c,r].
    # this is opposite of the [r,c] convntion thts more common
    rbins,cbins = alleyBins[alley-1]['rows'], alleyBins[alley-1]['cols']
    if rbins.shape[0] < cbins.shape[0]:
        hs = np.histogram2d(spikes[:,1],spikes[:,2],bins=[cbins,rbins])[0]
        ho = np.histogram2d(position[:,1],position[:,2],bins=[cbins,rbins])[0]
    else:
        hs = np.histogram2d(spikes[:,2],spikes[:,1],bins=[rbins,cbins])[0]
        ho = np.histogram2d(position[:,2],position[:,1],bins=[rbins,cbins])[0]
    n = (hs*np.reciprocal(ho))*33
    n = weird_smooth(n,2)
    n[np.where(ho==0)] = np.nan
    return n

## Diagnostic Plotting

In [34]:
from matplotlib.colors import LinearSegmentedColormap
def makeCustomColormap(nb=100,name='mymap',c=[]):
    if c ==[]:
        c = [(0,0,1),(1,1,0),(1,0,0)]
    mycm = LinearSegmentedColormap.from_list(name,c,N=nb)
    return mycm

In [711]:
rbinNum,cbinNum = 16,31
alleyBins = {i:{'rows':None,'cols':None} for i in range(17)}
for i,v in enumerate(alleyBounds.values()):
    x,y = v
    if (x[1]-x[0]) > (y[1]-y[0]):
        bins = [rbinNum,cbinNum]
    elif (y[1]-y[0]) > (x[1]-x[0]):
        bins = [cbinNum,rbinNum]
    else:
        print("error")
    alleyBins[i]['rows'] = np.linspace(alleyBounds[i][1][0], alleyBounds[i][1][1],num=bins[0])
    alleyBins[i]['cols'] = np.linspace(alleyBounds[i][0][0], alleyBounds[i][0][1],num=bins[1])

In [353]:
fig, ax = plt.subplots(6,3,figsize=(10,10))
axList = fig.axes
for alley in range(17):
    for txt,c in zip(['A', 'B', 'C'],['r','b','g']):
        visitSpikes, _ = linearizeVisits(alley+1, txt)
        avg = linearizeAlley(alley+1, txt,**{'nanpol':'nan'})
        err = sem(visitSpikes, axis=0,nan_policy='omit')
        axList[alley].plot(avg,f"{c}--")
        #axList[alley].fill_between(range(len(err)), avg+err, avg-err,color=f"{c}",alpha=0.5)
        axList[alley].set_title(f"Alley {alley+1}")

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in reciprocal
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in multiply
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Use

In [392]:
visitsSpikes,visitsPos = getVisits(13,'A')
numCols = 3
fig, ax = plt.subplots(3,4,figsize=(10,10))
axList = fig.axes
for i,(s,p) in enumerate(zip(visitsSpikes, visitsPos)):
    n=computeSingleRM(s,p,12)
    plt.figure(i+1)
    plt.imshow(n)
    

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in reciprocal
  from ipykernel import kernelapp as app
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
Traceback (most recent call last):
  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backends\backend_qt5agg.py", line 149, in __draw_idle_agg
    self.draw()
  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backends\backend_qt5agg.py", line 127, in draw
    super(FigureCanvasQTAggBase, self).draw()
  File "C:\Users\whockei1\Ap

  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\image.py", line 548, in draw
    renderer, renderer.get_image_magnification())
  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\image.py", line 774, in make_image
    unsampled=unsampled)
  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\image.py", line 370, in _make_image
    a_min = np.ma.min(A).astype(scaled_dtype)
  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\ma\core.py", line 3205, in astype
    if self._fill_value is not None:
AttributeError: 'MaskedConstant' object has no attribute '_fill_value'
Traceback (most recent call last):
  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\backends\backend_qt5agg.py", line 149, in __draw_idle_agg
    self.draw()
  File "C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib

## Generate data for 2d, 1d RMs and have cells with plots to compare them

In [745]:
alley,txt =12,'A'
s,p = getVisits(alley,txt)
Z = computeSingleRM(list_to_arrays(s),list_to_arrays(p),alley)
visitSpikes, visitOccs = linearizeVisits(alley, txt)
n = linearizeAlley(alley,txt,**{"nanpol":"nan"})

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in reciprocal
  from ipykernel import kernelapp as app
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app


In [572]:
# Plot avg of 2d RM and imshow the 2d rm
plt.figure(plt.gcf().number+1)
plt.plot(np.nanmean(Z,axis=getAxType(Z)))
plt.figure(plt.gcf().number+1)
plt.imshow(Z,origin='lower')
plt.title(f"Alley {alley} Texture {txt}")

Text(0.5,1,'Alley 12 Texture A')

In [573]:
# Plot avg of 2d RM along long dim, and plot avg of visits from the rms matrix
plt.figure(nf())
plt.plot(np.nanmean(Z,axis=getAxType(Z)),color='b',marker='^')
plt.plot(n,color='r',marker='^') # axis always 0 because rms matrix contructeed to by numVistis -by- longDim
plt.title(f"Alley {alley}, Texture {txt}. Avg of 2d RM (blue) and avg of visits (red). Omit nans")

Text(0.5,1,'Alley 12, Texture A. Avg of 2d RM (blue) and avg of visits (red). Omit nans')

## GridSpec-based 2D Track RM by txt

In [311]:
gs_lookup = {1:[0,1], 2:[2,1], 3:[1,0], 4:[1,2], 5:[0,3],
             6:[1,4], 7:[0,5], 8:[1,6], 9:[2,5], 10:[3,6],
             11:[4,5], 12:[3,4], 13:[2,3], 14:[4,3], 15:[3,2],
             16:[4,1], 17:[3,0]}
gs = GS.GridSpec(5,7)

In [574]:
txt = 'A'
gs = GS.GridSpec(5,7)
for i in range(1,18):
    r,c = gs_lookup[i]
    ax = plt.subplot(gs[r,c])
    s,p = getVisits(i,txt)
    if len(p) > 0:
        Z = computeSingleRM(list_to_arrays(s),list_to_arrays(p),i)
        ax.imshow(Z,origin='lower')
plt.suptitle(f"Collated Texture {txt}")

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in reciprocal
  from ipykernel import kernelapp as app
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


Text(0.5,0.98,'Collated Texture A')

# Organizing data structure
### Format unit - alley - visit - [spikes, pos, 2d rm, lin rm, metadata]

In [221]:
class UnitData():
    ''' 
    Class to store data for a single unit. 
    
    Data divided by alley in a dict. From there each visit
    is a subdict vals {spikes, pos, 2d rm, metadata}
    
    Data is either loaded from raw cl-files using behavioral
    parsing functions or it loads saved data from jsons depending
    on loadType value.
    
    Needed in namespace: 
    - alleyTracking, txtVisits, alleyVisits (if loading raw)
    - datafile
    - position
    
    '''
    def __init__(self, unitName, experimentCode, loadType='raw'):
        self.name = unitName
        self.experimentCode = experimentCode
        self.alleys = {i:[] for i in range(1,18)}
        
    def loadData_raw(self):
        '''Loads from parseBehavioral helper fx.
        Outputs of which must be in namespace'''
        self.computeAlleyBins()
        clust = util.read_clust(datafile+self.name)
        clust = np.asarray(clust)
        spikexy = util.getPosFromTs(clust,position)
        spikes = np.column_stack((clust,spikexy))
        spikes = spikes[np.where(spikes[:,2] > 50)]
        self.spikes = spikes
        self.linRMS = {i:{'A':None, 'B':None, 'C':None} for i in range(1,18)}
        self.alleyRMS = {i:{'A':None, 'B':None, 'C':None} for i in range(1,18)} # ie. 2d RM
        
        for alley in range(1,18):
            
            for visit in range(len(alleyVisits[alley-1])):
                visitsOcc = util.getVisitPos(alley-1, visit, ts, alleyVisits, position)
                visitsSpk = util.getVisitPos(alley-1, visit, self.spikes[:,0],alleyVisits, position)
                self.alleys[alley].append({'spikes': visitsSpk, 
                                       'occs': visitsOcc, 
                                       'ratemap': self.computeSingleRM(visitsSpk, visitsOcc, alley,dim=2),
                                       'metadata': self.generateVisitMetadata(alley, visit)
                                        })
                
        allS, allO = np.empty((0,3)), np.empty((0,3)) # for collecting all spikes/occs to make overall RM
        for alley in range(1,18):
            for txt in ['A', 'B', 'C']:
                s,p = self.collapseVisits(alley, txt)
                if len(s) > 0 and len(p) > 0:
                    s,p = util.list_to_arrays(s),util.list_to_arrays(p)
                    allS, allO = np.vstack((allS, s)), np.vstack((allO, p))
                    collapsed1DRateMap = self.computeSingleRM(s, p, alley,dim=1)
                    collapsed2DRateMap = self.computeSingleRM(s, p, alley,dim=2)
                    self.linRMS[alley][txt] = collapsed1DRateMap
                    self.alleyRMS[alley][txt] = collapsed2DRateMap
            self.alleyRMS[alley]['overall'] = self.computeSingleRM(allS, allO, alley,dim=2)
                
    def computeAlleyBins(self,longDimBins=30,shortDimBins=15):
        longDimBins += 1
        shortDimBins += 1
        self.alleyBins = {i:{'rows':None,'cols':None} for i in range(17)}
        for i,v in enumerate(alleyBounds.values()):
            x,y = v
            if (x[-1]-x[0]) > (y[-1]-y[0]):
                bins = [shortDimBins, longDimBins] #again, np.2dhist takes [x,y] which means [c, r]
            elif (y[-1]-y[0]) > (x[-1]-x[0]):
                bins = [longDimBins, shortDimBins]
            else:
                print("error")
            self.alleyBins[i]['rows'] = np.linspace(alleyBounds[i][1][0], alleyBounds[i][1][1],num=bins[0])
            self.alleyBins[i]['cols'] = np.linspace(alleyBounds[i][0][0], alleyBounds[i][0][1],num=bins[1])
            
    
                
    def computeSingleRM(self, spikes,position,alley,dim):
        '''Compute a single rate map from an array of spikes
        and pos of array form (ts,x,y). Not using track coords, just 
        0-max bin size in either array. Chooses which dim is long
        depending on which alley (lookup in fx)'''
        #So np.hist2d takes bins in [x,y] which is [c,r].
        # this is opposite of the [r,c] convntion thts more common
        rbins,cbins = self.alleyBins[alley-1]['rows'], self.alleyBins[alley-1]['cols']
        hs = np.histogram2d(spikes[:,2],spikes[:,1],bins=[rbins, cbins])[0]
        ho = np.histogram2d(position[:,2],position[:,1],bins=[rbins, cbins])[0]
        if dim == 2:
            n = (hs*np.reciprocal(ho))*33
            n = util.weird_smooth(n,2)
            n[np.where(ho==0)] = np.nan
        elif dim == 1:
            ls,lo  = np.sum(hs,axis=util.getAxType(ho)), np.sum(ho,axis=util.getAxType(ho))
            n = (ls* np.reciprocal(lo)) * 33  
            n = util.weird_smooth(n,2)
            n[np.where(lo==0)] = np.nan
        return n
    
    def collapseVisits(self, alley, txt):
        visitSpikes, visitPos = [], []
        for visit in self.alleys[alley]:
            if visit['metadata']['stimulus'] == txt:
                visitSpikes.append(visit['spikes'])
                visitPos.append(visit['occs'])
                
        return visitSpikes, visitPos
    
    def generateVisitMetadata(self, alley, visit):
        txt = txtVisits[alley-1][visit]
        return {"stimulus":txt}

In [5]:
class BehavioralData():
    """
    Class to read in position data from pos.p
    Also reads in behavior about alley visits and stimuli present.
    Does not store, just returns. Wrapper fx loadData.
    """
    def __init__(self,  datafile, experimentCode):
        self.datafile = datafile
        self.experimentCode = experimentCode
        
    def loadData(self):
        """
        Returns ts, position, alleyTracking, alleyVisits,  txtVisits
        """
        pos = util.read_pos(datafile)
        ts = np.asarray(sorted(list(pos.keys())))
        posx = [640 - pos[i][0] for i in ts]
        posy = [pos[i][1] for i in ts]
        for t in ts:  
            pos[t][0] = 640 - pos[t][0]
        position = np.column_stack((ts,posx,posy))
        position = position[np.where(position[:,2] > 50)]
        alleyTracking, alleyVisits,  txtVisits = pBehav.getDaysBehavioralData(datafile)
        return ts, position, alleyTracking, alleyVisits,  txtVisits

In [3]:
class BasicRateMaps():
    """
    Class with plotting fx for:
    - 2d ratemaps
    - 1d ratemaps (computed differently than 2d)
    - spike maps (spikes over trajectory)
    """
    def __init__(self):
        self.subplotColNum = 4
        self.currArrayMax = 0 # current maximum of arrays being plotted so that
                               # acoss-plot entites like colorbars can have a common scale
        self.rmDims = {'long':30, 'short':15} #todo: dont hardcode this. lookup on init from ratterdam_Defaults.
        self.gs_lookup = {1:[0,1], 2:[2,1], 3:[1,0], 4:[1,2], 5:[0,3],
                          6:[1,4], 7:[0,5], 8:[1,6], 9:[2,5], 10:[3,6],
                          11:[4,5], 12:[3,4], 13:[2,3], 14:[4,3], 15:[3,2],
                          16:[4,1], 17:[3,0]}
        self.colormap = self.makeCustomColormap()
        
    
    def makeCustomColormap(self, nb=100,name='mymap',c=[]):
        if c ==[]:
            c = [(0,0,0.5),(1,1,0),(1,0,0)]
        mycm = LinearSegmentedColormap.from_list(name,c,N=nb)
        return mycm
    
    def generateSingleSpikeMap(self, spikes, pos, alley, **kwargs):
        """
        Spike/Pos data n,3 array ts,x,y
        """
        self.ax.plot(pos[:,1], pos[:,2],'k',alpha=0.5)
        self.ax.scatter(spikes[:,1], spikes[:,2], c='r')
        self.drawAlleyBounds(self.ax, alley)
        self.annotate(**kwargs)
        
    def drawAlleyBounds(self, ax, alley, **kwargs):
        """
        Assume 1-idx entry
        """
        alley = alley - 1
        a = alleyBounds[alley]
        x1,y1,x2,y2 = a[0][0], a[1][0], a[0][1], a[1][1]
        for x,y in zip([[x1, x1], [x1, x2], [x2, x2], [x1, x2]], [[y1, y2], [y2, y2], [y1, y2], [y1, y1]]):
            ax.plot(x, y, 'k')
            
    def compute_AvgTrace(self, unit, alley, txt):
        """
        Creates a rate map for each visit and returns
        avg trace and sem trace
        
        Because defining linear RM by avg of visits
        is only done for visualization purposes, the code
        to compute it (this fx) is in the plotting class 
        rather than the unit data class
        """
        lins = np.empty((0, self.rmDims['long']))
        for visit in unit.alleys[alley]:
            if visit['metadata']['stimulus'] == txt:
                spk, occ = visit['spikes'], visit['occs']
                if occ.shape[0] > 0: #yes all visits should, by def, have occupancy but you never know
                    lin = unit.computeSingleRM(spk, occ, alley, dim=1)
                    lins = np.vstack((lins, lin))
                    
        avg = np.nanmean(lins, axis=0) 
        err = sem(lins, axis=0, nan_policy='omit')
        return avg, err        
            
    def annotate(self, **kwargs):
        if 'title' in kwargs:
            self.ax.set_title(kwargs['title'])
            
    def plot_SpikemapAllVisits(self, unit, alley):
        """
        Generates subplots each of which a spike map of a visit to alley
        
        Reminder on unit structure. unit.alleys[alley] is a list of dicts
        Ea. has spikes, occ, rm, metadata
        """
        alleydata = unit.alleys[alley]
        self.fig, self.axes = plt.subplots(int(np.ceil(len(alleydata)/self.subplotColNum)), self.subplotColNum)
        for i in range(len(alleydata)):
            self.ax = self.fig.axes[i]
            spikes = alleydata[i]['spikes']
            occs   = alleydata[i]['occs']
            txt    = alleydata[i]['metadata']['stimulus']
            print(spikes.shape)
            print(occs.shape)
            print("-----------")
            self.generateSingleSpikeMap(spikes, occs, alley, **{'title':txt})
            

    def getMaxArrays(self, unit, stim, cutoff = 0.98):
        """
        Compute maximum value of colormap for 2d ratemap
        This max defined as the firing rate which, at
        or below this value, accounts for cutoff proportion
        of the total firing rate occurances (discretized in 100 bins)
        Essentially the histogram bin that accounts for cutoff proportion
        of the cumulative sum of the bin counts.
        """
        frs = []
        
        #This conditional is here because I want
        # to scale alleys in 'overall' plots to one colorbar
        # scale and the alleys in plots for A, B, C to a separate
        # common-to-them colorbar. So grab different sets of arrays to get max
        if stim  == 'textures':
            arrayTypes = ['A', 'B', 'C']
        elif stim == 'overall':
            arrayTypes = ['overall']
            
        for arrayType in arrayTypes:
            for i in range(1,18):
                rm = unit.alleyRMS[i][arrayType]
                if type(rm) == np.ndarray:
                    rm = np.ndarray.flatten(rm)
                    if not all(np.isnan(rm)):
                        frs.extend(rm)
        frs = np.asarray(frs)
        frs = frs[np.isfinite(frs)]
        h,b = np.histogram(frs, bins=100)
        frcum = np.cumsum(h)
        propExp = np.asarray([i/h.sum() for i in frcum])
        try:
            thresh = np.where(propExp < cutoff)[0][-1]
        except:
            thresh = np.where(b == np.median(b))
        return b[thresh]*2.5
    
    
    def getMaxLins(self, unit):
        """
        Get maximum y limit for plotting linear ratemaps
        Logic is substantially different from the 2d version
        so it's its own fx.
        Scans all rms across all alleys, finds max and sets the 
        max to be like .2* higher than that. 
        """
        mymax = 0
        for alley in range(1,18):
            for txt in ['A', 'B', 'C']:
                avg, err = self.compute_AvgTrace(unit, alley, txt)
                if np.nanmax(avg+err) > mymax:
                    mymax = np.nanmax(avg+err)
        mymax = mymax*1.2
        return mymax
    
    
    def add_colorbar(self, fig, im):
        #left bottom width heigh
        cax = fig.add_axes([.2,.95,.6,.05])
        fig.colorbar(im, cax=cax, orientation='horizontal')
        
    def plot_2dAlleys(self, ax, unit, alley, data,**kwargs):
        '''Plot 2d (imshow) of alley
         Data == "overall" gives all visits
         Data == A,B,C gives you those visits, collapsed.
         If you say data == "texture" and dont give a kwarg['texture'] value ERROR
        '''
        im = False
        rm = unit.alleyRMS[alley][data]
        if type(rm) == np.ndarray:
            im = ax.imshow(rm, origin='lower', interpolation = 'None', vmax= kwargs['max'], cmap = self.colormap)
        
        # this ugly workaround is to get an imshow result
        # from at least one alley that i can use to
        # generate colorbar(). Wish i knew how to generate
        # a colobar without a plot 
        if kwargs and 'return' in kwargs:
            if kwargs['return'] == True:
                return im
            
    def plot_linAlley(self, ax, unit, alley, txt="All", **kwargs):
        """Will either just plot 1 txt if given
        or if txt='All' then all 3 traces. Traces
        are NOT linear RMS, but are avg/sem of individual trial
        RMs. For quantitative analyses eg perm tests
        lin RMS as defined by unit.linRMS are used. These
        are essentially collapsing acros trials to get trace. Here,
        and similar viz routines, use avging of trial rms. Methods yield different
        results. See Unit doc for details"""
        if txt.lower() == 'all':
            for txt,color in zip(['A', 'B', 'C'], ['r', 'b', 'g']):
                avg, err = self.compute_AvgTrace(unit, alley, txt)
                ax.plot(avg, f"{color}")
                ax.fill_between(range(len(avg)), avg-err, avg+err, color=f"{color}", alpha=0.5)
            ax.set_title(f"{alley}")
            if kwargs and 'ylim' in kwargs:
                ax.set_ylim([0, kwargs['ylim']])
        else:
            return "Further functionality not yet implemented. Please stick to defaults"
        
    def plot_2dWholeTrack(self, position, unit):
        """
        Will plot the whole track firing rate map.
        Does not take an axis arg - makes it's own sep figure.
        Does not break data into alleys. All spikes / all pos
        ratterdam_Defaults as bin sizes.
        Camera resolution of 640 (cols) x 480 (rows) is hardcoded.
        """
        fig, ax = plt.subplots(figsize=(10,10))
        col, row = np.linspace(0, 640, wholeAlleyBins[0]), np.linspace(0,480, wholeAlleyBins[1])
        hs = np.histogram2d(unit.spikes[:,2],unit.spikes[:,1],bins=[row, col])[0]
        ho = np.histogram2d(position[:,2],position[:,1],bins=[row, col])[0]
        n = (hs*np.reciprocal(ho))*33
        n = util.weird_smooth(n,1)
        n[np.where(ho==0)] = np.nan
        im = ax.imshow(n, aspect='auto', interpolation='None', origin='lower')
        self.add_colorbar(fig, im)
        ax.set_axis_off()
        #plt.tight_layout()
        ax.set_title(f"Overall ratemap for {unit.name}")

        
        

# Routine to Generate and Save Ratemaps for all clusters in a given day

In [59]:
exp = "DFD4"
dayCode = f"R765{exp}\\"
figpath = f"C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Figures\\R765{exp}\\velocityFiltered_{velocity_filter_thresh}\\"
if not os.path.isdir(figpath):
    os.mkdir(figpath)
datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
behav = core.BehavioralData(datafile, exp, velocity_filter_thresh)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()
plot = BasicRateMaps()
gs = GS.GridSpec(5,7)

for subdir, dirs, fs in os.walk(datafile):
    for f in fs:
        if 'cl-maze' in f and 'OLD' not in f and 'Undefined' not in f:
            clustname = subdir[subdir.index("TT"):] + "\\" + f
            print(clustname)
            unit = core.UnitData(clustname, datafile, "", alleyBounds, alleyVisits, txtVisits, position, ts)
            unit.loadData_raw()
            cn = clustname.split("\\")[0] + clustname.split("\\")[1]
            
            with PdfPages(figpath+cn+".pdf") as pdf:
                
                ###########################################
                # Whole track ratemap, no alley divisions #
                ###########################################
                
                plot.plot_2dWholeTrack(position, unit)
                try:
                    pdf.savefig()
                    print(f"Finished plot of 2d {stim} data")
                except:
                    print(f"Error for {stim}")
                plt.close()

                
                ##############################################################################
                # Stimulus Conditions A,B,C, (add overall to get overall, collapsed alleys) #
                #############################################################################
                
                mymax_txts = plot.getMaxArrays(unit, "textures") 
                mymax_overall = plot.getMaxArrays(unit, "overall")
                    
                for stim in ["A", "B", "C"]:
                    
                    if stim == "overall":
                        which_max = mymax_overall
                    else:
                        which_max = mymax_txts
                    
                    haveIM = False #Toggle for whether we've found a valid imshow we can use for the colorbar
                    fig = plt.figure(figsize=(10,10))
                    for row in [0, 1]:
                        for col in [0, 1, 2, 3]:
                            left =  (hw + vw)*col
                            alley = alleyLookup(row, col, "vertical")
                            bottom = voffset + (voffset*row) + (vh*(row))
                            ax = fig.add_axes([left, bottom, vw, vh])
                            
                            if not haveIM:
                                im = plot.plot_2dAlleys(ax, unit, alley, stim, **{"max":mymax, "return":True})
                                if im:
                                    haveIM = True
                            else:
                                plot.plot_2dAlleys(ax, unit, alley, stim, **{"max":mymax})
                            ax.set_xticks([])
                            ax.set_yticks([])

                    for row in [0, 1, 2]:
                        for col in [0, 1, 2]:
                            left = hoffset + (hoffset*col) + (hw*(col))
                            bottom = (hh + vh)*row
                            alley = alleyLookup(row, col, "horizontal")
                            ax = fig.add_axes([left, bottom, hw, hh])
                            if not haveIM:
                                im = plot.plot_2dAlleys(ax, unit, alley, stim, **{"max":mymax, "return":True})
                                if im:
                                    haveIM = True
                            else:
                                plot.plot_2dAlleys(ax, unit, alley, stim, **{"max":mymax})
                            ax.set_xticks([])
                            ax.set_yticks([])
                            
                    if im:
                        plot.add_colorbar(fig, im)
                    plt.suptitle(f"{stim}", fontsize=17)
                    try:
                        pdf.savefig()
                        print(f"Finished plot of 2d {stim} data")
                    except:
                        print(f"Error for {stim}")
                    plt.close()
                    
                ##########################
                # Linearized Rate Maps   #
                ##########################
                ylims = []
                fig, axes = plt.subplots(5,4, figsize=(10,10))
                for i in range(17):
                    ax = fig.axes[i]
                    plot.plot_linAlley(ax, unit,i+1)
                    ylims.append(ax.get_ylim()[1]) # get all max ylims of plots
                    
                # we don't know what all the ylims are until we calc and plot
                # so loop back over and adjust everything
                ymax = max(ylims)
                for i in range(17):
                    ax = fig.axes[i]
                    ax.set_ylim([0, ymax])

                pdf.savefig()
                plt.close()
                print("Finished linear RM subplots")
                

TT14\cl-maze1.1


C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: divide by zero encountered in reciprocal
  def generateVisitMetadata(self, alley, visit):
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: invalid value encountered in multiply
  def generateVisitMetadata(self, alley, visit):
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:312: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: divide by zero encountered in reciprocal
  
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: invalid value encountered in multiply
  
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:308: RuntimeWarning: invalid value encountered in multiply
  W=0*U.copy()+1


Finished plot of 2d overall data
Finished plot of 2d A data
Finished plot of 2d B data
Finished plot of 2d C data


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: Mean of empty slice
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Finished linear RM subplots
TT14\cl-maze1.2
Finished plot of 2d overall data
Finished plot of 2d A data
Finished plot of 2d B data
Finished plot of 2d C data
Finished linear RM subplots
TT14\cl-maze1.3
Finished plot of 2d overall data
Finished plot of 2d A data
Finished plot of 2d B data
Finished plot of 2d C data
Finished linear RM subplots
TT14\cl-maze1.4
Finished plot of 2d overall data
Finished plot of 2d A data
Finished plot of 2d B data
Finished plot of 2d C data
Finished linear RM subplots
TT14\cl-maze1.5
Finished plot of 2d overall data
Finished plot of 2d A data
Finished plot of 2d B data
Finished plot of 2d C data
Finished linear RM subplots
TT14\cl-maze1.6
Finished plot of 2d overall data
Finished plot of 2d A data
Finished plot of 2d B data
Finished plot of 2d C data
Finished linear RM subplots
TT4\cl-maze1.1
Error for overall
Error for A
Error for B
Error for C
Finished linear RM subplots
TT4\cl-maze1.10
Finished plot of 2d overall data
Finished plot of 2d A data
Finished 

## Tweaking different ratemap features

In [44]:
exp = "DFD4"
dayCode = f"R765{exp}\\"
figpath = f"C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Figures\\R765{exp}\\"
datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
behav = core.BehavioralData(datafile, exp, velocity_filter_thresh)
clustList = util.getClustList(datafile)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()
gs = GS.GridSpec(5,7)
unit = core.UnitData("TT7\\cl-maze1.3", datafile, "", alleyBounds, alleyVisits, txtVisits, position, ts)
unit.loadData_raw()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:120: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:120: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:312: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:125: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:125: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33


In [35]:
plot = BasicRateMaps()
stim = 'C'
mymax = plot.getMaxArrays(unit)
haveIM = False #Toggle for whether we've found a valid imshow we can use for the colorbar

fig = plt.figure(figsize=(10,10))
for i in range(1,18):
    r,c = plot.gs_lookup[i]
    ax = plt.subplot(gs[r,c])
    if not haveIM:
        im =  plot.plot_2dAlleys(ax, unit, i, stim, **{'max':mymax, 'return':True})

        if im is not False:
            haveIM = True  #So once we get a valid imshow, stop looking.
    else:
        plot.plot_2dAlleys(ax, unit, i, stim, **{'max':mymax})
    ax.set_title(f"{i}")
if im:
    plot.add_colorbar(fig, im)
plt.suptitle(f"{stim}", fontsize=17)

Text(0.5,0.98,'C')

In [60]:
%qtconsole --style paraiso-dark

In [55]:
reload(pBehav)
reload(core)
reload(Filt)

<module 'ratterdam_DataFiltering' from 'C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Code\\ratterdam_DataFiltering.py'>

In [156]:
hs = np.histogram2d(unit.spikes[:,2],unit.spikes[:,1],bins=[row, col])[0]
ho = np.histogram2d(position[:,2],position[:,1],bins=[row, col])[0]
n = (hs*np.reciprocal(ho))*33
n = util.weird_smooth(n,1)
n[np.where(ho==0)] = np.nan
plt.imshow(n, aspect='auto', interpolation='None', origin='lower')

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in reciprocal
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:312: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [181]:
ylims = []
fig, axes = plt.subplots(5,4, figsize=(10,10))
for i in range(17):
    ax = fig.axes[i]
    plot.plot_linAlley(ax, unit,i+1)
    ylims.append(ax.get_ylim()[1]) # get all max ylims of plots

# we don't know what all the ylims are until we calc and plot
# so loop back over and adjust everything
ymax = max(ylims)
for i in range(17):
    ax = fig.axes[i]
    ax.set_ylim([0, ymax])

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: divide by zero encountered in reciprocal
  
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: invalid value encountered in multiply
  
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:312: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: Mean of empty slice
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\whockei1\AppData\Loc